## Lab9 UCM EN RAG TTS Demo

In [17]:
import os

print(os.environ['HSA_OVERRIDE_GFX_VERSION'])

In [18]:
import os
os.environ['HIP_VISIABLE_DEVICES'] = "0"

In [19]:
import torch

In [20]:
print(torch.__version__)

2.4.0.dev20240401+rocm6.0


In [21]:
# Query GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    print('Using GPU:', torch.cuda.get_device_name(0))
    print('GPU properties:', torch.cuda.get_device_properties(0))
else:
    device = torch.device("cpu")
    print('Using CPU')

Using GPU: AMD Radeon PRO W7900
GPU properties: _CudaDeviceProperties(name='AMD Radeon PRO W7900', major=11, minor=0, gcnArchName='gfx1100', total_memory=46064MB, multi_processor_count=48)


In [22]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.

import nest_asyncio

nest_asyncio.apply()

In [23]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.node_parser import SentenceSplitter

from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

# Set embedding model
# Please download it ahead running this lab by "ollama pull nomic-embed-text"
#Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text")
Settings.embed_model = OllamaEmbedding(model_name="mxbai-embed-large")

# Set ollama model
Settings.llm = Ollama(model="llama3", request_timeout=120.0)

In [24]:
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

if not os.path.exists("./chroma_db/CUM_en_db"):
    # initialize client
    db = chromadb.PersistentClient(path="./chroma_db/CUM_en_db")
    # get collection
    chroma_collection = db.get_or_create_collection("CUM_en_db")
    # assign chroma as the vector_store to the context
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    # Load data
    documents = SimpleDirectoryReader(input_files=["../data/FordUM.pdf"]).load_data()
    # Build vector index per-document
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
        transformations=[SentenceSplitter(chunk_size=1024, chunk_overlap=50)],
    )
else:
    # initialize client
    db = chromadb.PersistentClient(path="./chroma_db/CUM_en_db")
    # get collection
    chroma_collection = db.get_or_create_collection("CUM_en_db")
    # assign chroma as the vector_store to the context
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    # load your index from stored vectors
    index = VectorStoreIndex.from_vector_store(
        vector_store, storage_context=storage_context
    )

In [25]:
# create a query engine
query_engine = index.as_query_engine(streaming=True, response_mode="compact", similarity_top_k=3)

In [26]:
# Query Test 0
response = query_engine.query("Which pages could get the detials about Exterior Lighting Control?")
response.print_response_stream()

Based on my analysis of the provided context information, I would recommend checking pages 657 and/or page 2 of the file ../data/FordUM.pdf for details about Exterior Lighting Control.

In [27]:
# Q
response = query_engine.query("What's the steps to SWITCHING THE DAYTIME RUNNING LAMPS ON AND OFF and which pages show it?")
response.print_response_stream()

According to the provided context information, you can access the vehicle drawer on the touchscreen, press "CONTROLS", then press "Zone Lighting". The page label for this is 117.

%pip install ChatTTS

In [28]:
import ChatTTS
import torchaudio
from IPython.display import Audio

In [29]:
chat = ChatTTS.Chat()
chat.load(compile=False) # Set to True for better performance

use default LlamaModel for importing TELlamaModel error: No module named 'transformer_engine'


True

In [30]:
print(str(response))

According to the provided context information, you can access the vehicle drawer on the touchscreen, press "CONTROLS", then press "Zone Lighting". The page label for this is 117.


In [31]:
wavs = chat.infer(str(response))

#torchaudio.save("output1.wav", torch.from_numpy(wavs[0]), 24000)

found invalid characters: {'7', '"', '1'}
text:  15%|████████████████████▋                                                                                                                    | 58/384(max) [00:00, 100.86it/s]
code:  23%|███████████████████████████████▏                                                                                                       | 473/2048(max) [00:04, 107.29it/s]


In [32]:
Audio(wavs[0], rate=24_000, autoplay=True)

## To-Do: Optimization the RAG

Refer to Basic Strategies: https://docs.llamaindex.ai/en/v0.10.19/optimizing/basic_strategies/basic_strategies.html